# LIBRARIES

In [18]:
import pandas as pd
import numpy as np
from pkg.constants import *
from pkg.greedy import greedy
from pkg.evaluation import Evaluation
from time import time



# LECTURA DE DATOS

In [19]:
deltas_df   = pd.read_csv(DELTAS_5M_PATH) 
deltas_matrix   = deltas_df.to_numpy()

# SOLUCIÓN GREEDY

## GENERA SOLUCION 

In [20]:
initial_state = deltas_matrix[0,:]
greedy_sol = greedy(initial_state)
greedy_sol.sum()

220.0

## EVALUA SOLUCION

In [21]:
# 574.5513854177129

evaluation = Evaluation()

evaluation.evaluate(greedy_sol)

574.5513854177129

# BUSQUEDA ALEATORIA

### GENERACIÓN DE SEMILLAS
Las semillas deben guardarse para poder repetir las ejecuciones y compaar resul

In [25]:
seeds = np.random.randint(0,99999,5)
print(seeds)

[88050 30580 60841  5078 35366]


### GENERAR SOLUCIONES VIABLES

In [26]:
def generate_solution(vector, stations, limit_capacity):
    capacity = vector.copy()

    capacity = np.round(capacity * limit_capacity / np.sum(capacity))
    total_gaps = np.sum(capacity)

    if total_gaps > limit_capacity:
        greater_station = np.argmax(capacity)
        capacity[greater_station] -= total_gaps - limit_capacity
    
    return capacity

In [28]:
# Generamos BA_TOTAL_ITERATIONS vectores de capacidad
stations = deltas_matrix.shape[1]

np.random.seed(seeds[0])
random_sequence = np.random.randint(2,10,stations * (BA_TOTAL_ITERATIONS))

# cada fila de capacities es una posible solucion de la BA
capacities = np.zeros((BA_TOTAL_ITERATIONS,stations))

# Relleneamos capacities con soluciones viables
initial, leap = 0, stations

control = np.zeros(BA_TOTAL_ITERATIONS)

for i in range(0,BA_TOTAL_ITERATIONS):

    # Rellenamos la fila con los numeros aleatorios
    vector = random_sequence[initial:initial+leap].copy()
    initial +=leap

    capacities[i,:] = generate_solution(
        vector         = vector,
        stations       = stations,
        limit_capacity = MAX_CAPACITY
    )
    
    control[i] = np.sum(capacities[i,:])



### ALGORITMO

In [32]:
initial_solution = capacities[0]
current_solution = initial_solution
best_solution = current_solution

best_solution_cost = evaluation.evaluate(best_solution)

for i in range(1,BA_TOTAL_ITERATIONS):
    current_solution = capacities[i]
    current_solution_cost = evaluation.evaluate(current_solution)

    if current_solution_cost < best_solution_cost:
        best_solution      = current_solution
        best_solution_cost = current_solution_cost

print(best_solution)
print(best_solution_cost)
print(np.sum(best_solution))

# [16.  9. 12. 16. 16. 19. 14. 12. 12. 16. 19. 21.  7.  9.  7. 14.]

[16.  9. 12. 16. 16. 19. 14. 12. 12. 16. 19. 21.  7.  9.  7. 14.]
437.7384250726773
219.0


# BUSQUEDA LOCAL

# ANOTACIONES DIA 21/03/2022
Dia 4: defensa de la práctica
la bsuqueda  tabu deberá tardar más o menos lo que tarda el enfriamiento simulado.
No tenemos en cuenta si los vecinos se repiten o no (no lo combprobamos.)
velocidad de movimiento: 5 (nos dice cuantas posibilidades hay)
matriz de frecuencias: como se hace??? 
greedy: queremos hacer algo inversamente prorporcional a lo que ha salido (hacemos la inversa de cada casilla de la matriz de frecuencias???) y luego normalizo (?)
ruleta de probabilidad inversa -> buscar libreria, que seguro que está hecho en algun lado. Si no, está el pseudo en la práctica.

la condicion de parada: que sea mayor o igual al numero de iteraciones 
generar vecinos tiene la restriccion tabu
cogemos el algoritmo de busqueda local y vmos añadiendo trocitos
la parte que suele generar más dudas suele ser la parte del greedy
